In [1]:
import numpy as np
import pandas as pd
import xarray as xr

from pointprocess import *
from lightning_setup import *

In [2]:
city = 'cedar'
c = Region(city=cities[city])
c.define_grid(step=1, units='km')

In [3]:
%%time
def get_FC(region, y):
    # filter CG using the flag in recent data, or by taking only strikes with
    # amblitude less than a value, or less than one value and greater than another
    ds = region.get_ds(y=y, filter_CG=dict(method='less_than', amax=-10), func=None)
    df = ds.to_dataframe()
    ds.close()
    df.index = df.time
    FC = df['lat'].resample('24H', base=12, label='right').count()
    FC.name = 'FC'
    return FC

FC = get_FC(c, 1996)
for y in range(1997,2017):
    FC = pd.concat([FC, get_FC(c, y)])

CPU times: user 5min 4s, sys: 4.93 s, total: 5min 9s
Wall time: 5min 4s


In [34]:
FC.sort_values(ascending=False, inplace=True)
FC.head(10)

time
2013-08-18 12:00:00    30330
1997-09-09 12:00:00    27404
1998-07-20 12:00:00    27192
2001-08-13 12:00:00    24381
1998-08-31 12:00:00    24175
2001-08-20 12:00:00    24130
2003-08-15 12:00:00    23857
2003-08-16 12:00:00    23339
2001-08-09 12:00:00    23240
2012-08-22 12:00:00    22128
Name: FC, dtype: int64

In [35]:
top_50 = FC.head(50)
top_200 = FC.head(200)
top_200.to_csv("Cedar_top_200_days_2010_2016.csv")

Smaller area flash count

In [36]:
EPK = Region(city=cities['cedar'])
EPK.SUBSETTED = False
EPK.CENTER = (37.7, -111.8)
EPK.RADIUS = 0.7
EPK.define_grid(step=1, units='km')

In [39]:
%%time
FC = get_FC(EPK, 1996)
for y in range(1997,2017):
    FC = pd.concat([FC, get_FC(EPK, y)])

CPU times: user 5min 50s, sys: 10.3 s, total: 6min
Wall time: 5min 45s


In [40]:
FC.sort_values(ascending=False, inplace=True)
FC.head(10)

time
2000-09-30 12:00:00    4126
2001-08-22 12:00:00    3916
2013-08-18 12:00:00    3406
2006-07-22 12:00:00    3269
2010-08-05 12:00:00    3147
2001-08-09 12:00:00    3085
1997-09-03 12:00:00    2912
1997-10-03 12:00:00    2732
1997-09-12 12:00:00    2507
2002-08-20 12:00:00    2489
Name: FC, dtype: int64

Convert to density:

In [41]:
EPK.ngrid_cells = (EPK.gridx.size-1) * (EPK.gridy.size-1)

flash_density = FC/float(EPK.ngrid_cells)